In [9]:
import seaborn as sns
import pandas as pd

df = pd.read_csv("/config/workspace/Dataset/tips.csv")

df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [10]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [12]:
df['sex'].unique()

array(['Female', 'Male'], dtype=object)

In [13]:
df['time'].unique()

array(['Dinner', 'Lunch'], dtype=object)

In [14]:
df['day'].unique()

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [15]:
df['smoker'].unique()

array(['No', 'Yes'], dtype=object)

In [16]:
df['sex']=df['sex'].replace({'Female':0,'Male':1})
df['smoker']=df['smoker'].replace({'No':0,'Yes':1})
df['day']=df['day'].replace({'Thur':0,'Fri':1,'Sat':2,'Sun':3})
df['time']=df['time'].replace({'Lunch':0,'Dinner':1})

In [17]:
df['sex']=df['sex'].astype(int)
df['smoker']=df['smoker'].astype(int)
df['day']=df['day'].astype(int)
df['time']=df['time'].astype(int)

In [18]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0,0,3,1,2
1,10.34,1.66,1,0,3,1,3
2,21.01,3.50,1,0,3,1,3
3,23.68,3.31,1,0,3,1,2
4,24.59,3.61,0,0,3,1,4
...,...,...,...,...,...,...,...
239,29.03,5.92,1,0,2,1,3
240,27.18,2.00,0,1,2,1,2
241,22.67,2.00,1,1,2,1,2
242,17.82,1.75,1,0,2,1,2


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    int64  
 3   smoker      244 non-null    int64  
 4   day         244 non-null    int64  
 5   time        244 non-null    int64  
 6   size        244 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 13.5 KB


In [20]:
X= df.drop(columns = ['time'])
y = df['time']

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=0)
X_train.shape, X_test.shape

((183, 6), (61, 6))

In [28]:
from sklearn.linear_model  import LogisticRegression

In [29]:
from sklearn.model_selection import GridSearchCV
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# parameter grid
parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}


In [30]:
logreg = LogisticRegression()
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',        # metric for scoring
                   cv=10)                     # number of folds

clf.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='accuracy')

In [31]:
clf.best_params_

{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}

In [32]:
clf.best_score_

0.9891812865497076

In [33]:
y_pred = clf.predict(X_test)

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [35]:
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat

array([[11,  5],
       [ 2, 43]])

In [36]:
true_positive = conf_mat[0][0]
false_positive = conf_mat[0][1]
false_negative = conf_mat[1][0]
true_negative = conf_mat[1][1]

In [37]:
Accuracy = (true_positive + true_negative) / (true_positive +false_positive + false_negative + true_negative)
Accuracy

0.8852459016393442

In [38]:
Precision = true_positive/(true_positive+false_positive)
Precision

0.6875

In [39]:
Recall = true_positive/(true_positive+false_negative)
Recall

0.8461538461538461

In [ ]:
import pickle
file = open('/config/workspace/Model/modelForPrediction.pkl','wb')
pickle.dump(logreg,file)
file.close()